In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)


from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split

s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'train.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
project = pd.read_csv(file_content_stream)
project.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [2]:
#Dropping na values
project = project.dropna()

In [3]:
df = list()
for i in range(1,100):
    ## Defining the input and taregt variables
    x= project[['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemsVoids', 'scansWithoutRegistration', 'quanitityModification', 'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition']]
    y= project['fraud']

    #Splitting data into train and test
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2)
    
    #Estimating lambda for lasso
    lasso_cv= LassoCV(alphas= [.001, .01, .1, 1, 10, 100], normalize= True, cv=5).fit(x_train, y_train)

    #Extracting best lambda
    cv_lambda= lasso_cv.alpha_
    #print('Estimated lambda for the lasso model is:', cv_lambda)

    #Building lasso
    lasso_md= Lasso(alpha= cv_lambda, normalize= True).fit(x_train, y_train)
    df.append(lasso_md.coef_)

In [4]:
A = pd.DataFrame(df)
A

,0,1,2,3,4,5,6,7,8
0,-0.020975,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
1,-0.015919,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
2,-0.018349,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
3,-0.021680,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
4,-0.020355,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...
94,-0.020766,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
95,-0.018626,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
96,-0.018948,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
97,-0.017779,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
